In [1]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import function_library_assignment_2 as fnc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.precision', 2)
pd.options.display.float_format = '{:.2f}'.format

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4
plt.rcParams['axes.axisbelow'] = True

In [2]:
wind_hour = 31
n_bus = 24
gen_data = fnc.read_data('gen_data')
system_demand = fnc.read_data('system_demand')['System Demand']
load_distribution = fnc.read_data('load_distribution')
gen_data = fnc.read_data('gen_data')[['Unit #', 'Node', 'P max MW', 'R+ MW','R- MW']] #"The commitment and ramping constraints and costs of all generators can be neglected."
gen_costs = fnc.read_data('gen_costs')
wind_data = fnc.read_data('wind_data', wind_hour=wind_hour, wind_scenarios=np.arange(100)) #The ED is only for a single hour - in this case, we choose hour 31

In [3]:
gen_costs

,Unit #,C ($/MWh),Cu ($/MWh),Cd ($/MWh),C+($/MWh),C-($/MWh)
0,1,13.32,15.00,14.00,15.00,11.00
1,2,13.32,15.00,14.00,15.00,11.00
2,3,20.70,10.00,9.00,24.00,16.00
3,4,20.93,8.00,7.00,25.00,17.00
4,5,26.11,7.00,5.00,28.00,23.00
5,6,10.52,16.00,14.00,16.00,7.00
6,7,10.52,16.00,14.00,16.00,7.00
7,8,6.02,0.00,0.00,0.00,0.00
8,9,5.47,0.00,0.00,0.00,0.00
9,10,0.00,0.00,0.00,0.00,0.00


In [4]:
wind_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,Expected
Wind Farm,,,,,,,,,,,,,,,,,,,,,
1,129.71,167.67,56.95,57.28,77.72,130.53,88.15,54.77,88.26,134.49,...,72.38,83.37,49.65,133.94,145.62,104.54,57.31,67.36,66.15,87.84
2,149.91,81.91,21.47,54.45,36.67,64.63,81.76,82.92,118.74,129.80,...,76.48,43.40,54.13,65.01,119.32,93.82,47.16,76.89,52.26,84.95
3,140.28,145.61,35.52,82.77,22.09,151.28,78.23,159.62,119.69,85.96,...,73.25,62.76,152.92,76.48,109.01,62.66,40.34,159.71,39.86,95.18
4,137.33,91.46,68.42,36.10,59.74,82.04,66.49,104.59,126.92,157.77,...,52.61,47.74,119.34,59.24,180.08,45.01,57.55,105.68,68.38,94.18
5,63.42,58.30,85.68,30.09,52.93,28.73,20.65,40.39,50.97,47.52,...,29.54,38.12,20.28,37.58,36.29,58.14,29.18,20.20,33.77,53.94
6,78.10,66.57,101.99,3.27,21.48,0.90,71.20,34.24,41.17,128.50,...,45.18,23.02,61.70,100.68,26.37,41.55,52.85,57.98,11.43,54.71


Assumption: Based on the exercises, we set the day-ahead cost of the wind farm to be 5% of the average cost of the gens and the regulation cost to be 10% of the average regulation cost of the gens

In [5]:
wf_costs = gen_costs.iloc[0:6].copy()
wf_costs['C ($/MWh)'] = 0.05 * gen_costs['C ($/MWh)'].mean()
wf_costs['C+($/MWh)'] = 0.1 * gen_costs['C+($/MWh)'].mean()
wf_costs['C-($/MWh)'] = 0.1 * gen_costs['C-($/MWh)'].mean()
wf_costs['Cu ($/MWh)'] = 0.1 * gen_costs['Cu ($/MWh)'].mean()
wf_costs['Cd ($/MWh)'] = 0.1 * gen_costs['Cd ($/MWh)'].mean()
wf_costs

,Unit #,C ($/MWh),Cu ($/MWh),Cd ($/MWh),C+($/MWh),C-($/MWh)
0,1,0.62,1.00,0.89,1.41,0.90
1,2,0.62,1.00,0.89,1.41,0.90
2,3,0.62,1.00,0.89,1.41,0.90
3,4,0.62,1.00,0.89,1.41,0.90
4,5,0.62,1.00,0.89,1.41,0.90
5,6,0.62,1.00,0.89,1.41,0.90


# Task 3b

In [6]:
t = 0 #hour
demand = system_demand[t]

n_gen = len(gen_data.index)
n_wf = len(wind_data.index)
p_max_wf = 300

In [7]:
n_gen

12

### Setup normal distribution

We normalize the wind power data

In [8]:
pd.set_option('display.precision', 5)
pd.options.display.float_format = '{:.5f}'.format

In [9]:
wind_data = wind_data / p_max_wf

In [10]:
cov = wind_data[wind_data.columns[:-1]].T.cov().values
wind_data[wind_data.columns[:-1]].T.cov()

Wind Farm,1,2,3,4,5,6
Wind Farm,,,,,,
1,0.01114,0.00760,0.00724,0.00485,0.00019,0.00162
2,0.00760,0.01642,0.00776,0.00657,0.00204,0.00253
3,0.00724,0.00776,0.01842,0.01050,0.00036,0.00167
4,0.00485,0.00657,0.01050,0.01491,0.00163,0.00235
5,0.00019,0.00204,0.00036,0.00163,0.00567,0.00020
6,0.00162,0.00253,0.00167,0.00235,0.00020,0.01392


In [11]:
mu = wind_data[wind_data.columns[:-1]].T.mean().values
mu

array([0.29278438, 0.28316482, 0.31727709, 0.31392262, 0.17980337,
       0.18237634])

The inverse CDF of a STANDARD normal distribution

In [12]:
from scipy.stats import norm
epsilon = 0.1
norm.ppf(1 - epsilon)

1.2815515655446004

### Setup model based on Exercise 12 solution and solve

In [13]:
# Define ranges and hyperparameters  
CONTROLLABLE_GENERATORS = [i for i in range(12)] #range of controllable generators
WIND_GENERATORS = [i for i in range(12, 18)] #range of wind generators
GENERATORS = [i for i in range(18)] #range of all generators
LOAD = system_demand[t] #system load

# Set values of input parameters
dispatch_cost = {} # Generation costs in DKK/MWh
reserve_cost_up = {} # costs for upward reserve in DKK/MW
reserve_cost_down = {} # costs for downward reserve in DKK/MW
adjustment_cost_up = {} # costs for upward adjustments in real time in DKK/MWh
adjustment_cost_down = {} # costs for downward adjustments in real time in DKK/MWh
generation_capacity = {} # Generators capacity (Q^G_i) in MW
adjustment_capacity_up = {} # upward adjustment capacity (Q^up_i) in MW
adjustment_capacity_down = {} # downward adjustment capacity (Q^dw_i) in MW
wind_availability_scenario = {} # scenarios of available wind production -
wind_availability_expected = {}
wind_availability_standard_deviation = {}
wind_availability_min ={} # min available wind production (normalized)
wind_availability_max = {} # max available wind production (normalized)

for g in GENERATORS:
    if g < 12:
        dispatch_cost[g] = gen_costs['C ($/MWh)'][g]
        reserve_cost_up[g] = gen_costs['Cu ($/MWh)'][g]
        reserve_cost_down[g] = gen_costs['Cd ($/MWh)'][g]
        adjustment_cost_up[g] = gen_costs['C+($/MWh)'][g]
        adjustment_cost_down[g] = gen_costs['C-($/MWh)'][g]
        generation_capacity[g] = gen_data['P max MW'][g]
        adjustment_capacity_up[g] = gen_data['R+ MW'][g]
        adjustment_capacity_down[g] = gen_data['R- MW'][g]

    else:
        dispatch_cost[g] = wf_costs['C ($/MWh)'][g - 12] # Generation costs in DKK/MWh
        reserve_cost_up[g] = wf_costs['Cu ($/MWh)'][g - 12]
        reserve_cost_down[g] = wf_costs['Cd ($/MWh)'][g - 12]
        adjustment_cost_up[g] = wf_costs['C+($/MWh)'][g - 12]
        adjustment_cost_down[g] = wf_costs['C-($/MWh)'][g - 12]
        generation_capacity[g] = p_max_wf
        adjustment_capacity_up[g] = p_max_wf
        adjustment_capacity_down[g] = p_max_wf

wind_availability_expected = wind_data['Expected'].values

wind_availability_standard_deviation = np.diag(cov)

In [14]:
def _solve_reserve_dimensioning_model_():
    
    # Create a Gurobi model for the optimization problem
    DA_model = gb.Model(name='Day-ahead economic dispatch and reserve dimensioning problem')
        
        
    # Set time limit
    DA_model.Params.TimeLimit = 500
    
    # Add variables to the Gurobi model
    # first-stage variables

    generator_dispatch = {} # electricity production of generators      
    generator_reserve_up = {} # upward reserves of generators        
    generator_reserve_down = {} # downward reserves of generators      
    
    for g in CONTROLLABLE_GENERATORS:
        generator_dispatch[g] = DA_model.addVar(lb=0,ub=generation_capacity[g],name='Dispatch of generator {0}'.format(g))
        generator_reserve_up[g] = DA_model.addVar(lb=0,ub=adjustment_capacity_up[g],name='Upward reserve of generator {0}'.format(g))
        generator_reserve_down[g] = DA_model.addVar(lb=0,ub=adjustment_capacity_down[g],name='Downward reserve of generator {0}'.format(g))

    for w in WIND_GENERATORS:
        generator_dispatch[w] = DA_model.addVar(lb=0,ub=generation_capacity[w],name='Dispatch of wind farm {0}'.format(w - 12))
        generator_reserve_up[w] = DA_model.addVar(lb=0,ub=adjustment_capacity_up[w],name='Upward reserve of wind farm {0}'.format(w - 12))
        generator_reserve_down[w] = DA_model.addVar(lb=0,ub=adjustment_capacity_down[w],name='Downward reserve of wind farm {0}'.format(w - 12))

    # linear decision rules for second-stage variables
    generator_adjustment_up_intersect = {g:DA_model.addVar(lb=-gb.GRB.INFINITY,name='upward adjustment LDR parameter of generator {0} '.format(g)) for g in GENERATORS} # electricity production adjustment of generators in real time (\Delta x^G_i)
    generator_adjustment_down_intersect = {g:DA_model.addVar(lb=-gb.GRB.INFINITY,name='downward adjustment LDR parameter of generator {0} '.format(g)) for g in GENERATORS} # electricity production adjustment of generators in real time (\Delta x^G_i)
    generator_adjustment_up = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY,name=('upward adjustment LDR parameter (%d) of generator %d' % (w, g))) for g in GENERATORS} for w in range(n_wf)]
    generator_adjustment_down = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY,name=('downward adjustment LDR parameter (%d) of generator %d' % (w, g))) for g in GENERATORS} for w in range(n_wf)] 

    #auxiliary variables for SOC constraints (non-negative)
    y = [{g:DA_model.addVar(lb=0) for g in GENERATORS} for n in range(6)] #RHS auxiliary variables - we have 6 chance constraints

    #Constructing the LHS vectors associated with each chance constraint
    x_1 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in GENERATORS} for w in range(n_wf)]
    x_2 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in GENERATORS} for w in range(n_wf)]
    x_3 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in GENERATORS} for w in range(n_wf)]
    x_4 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in GENERATORS} for w in range(n_wf)]
    x_5 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in GENERATORS} for w in range(n_wf)]
    x_6 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in CONTROLLABLE_GENERATORS} for w in range(n_wf)]
    x_7 = [{g:DA_model.addVar(lb=-gb.GRB.INFINITY) for g in WIND_GENERATORS} for w in range(n_wf)]
     
    
    # update gurobi model
    DA_model.update()
    
    # Set objective function and optimization direction of the Gurobi model
    total_cost = gb.quicksum(dispatch_cost[g] * generator_dispatch[g] + reserve_cost_up[g] * generator_reserve_up[g] + reserve_cost_down[g] * generator_reserve_down[g] 
                             + adjustment_cost_up[g] * generator_adjustment_up_intersect[g] - adjustment_cost_down[g] * generator_adjustment_down_intersect[g] 
                             + gb.quicksum(wind_availability_expected[w] * (adjustment_cost_up[g] * generator_adjustment_up[w][g] 
                                                                            - adjustment_cost_down[g] * generator_adjustment_down[w][g]) for w in range(n_wf))
                             for g in GENERATORS) # expected electricity production cost
    
    DA_model.setObjective(total_cost, gb.GRB.MINIMIZE) #minimize cost

    # Add constraints to the Gurobi model
    # DA balance equation (Eq. 34 in report)
    DA_balance_constraint = DA_model.addConstr(gb.quicksum(generator_dispatch[g] for g in GENERATORS) == LOAD, name='Day-ahead balance equation')
 

    # DA_dispatch_min_constraint (Eq. 40 and 41 in report)
    DA_dispatch_min_constraint = {g:DA_model.addLConstr(generator_dispatch[g] - generator_reserve_down[g], 
                                                       gb.GRB.GREATER_EQUAL,
                                                       0,name='day-ahead dispatch and reserved capacity lower bound') for g in GENERATORS}
    
    # DA_dispatch_max_constraint for synchronous generators (Eq. 39 in report)
    DA_dispatch_max_constraint = {}
    for g in CONTROLLABLE_GENERATORS:
        DA_dispatch_max_constraint[g] = DA_model.addLConstr(generator_dispatch[g] + generator_reserve_up[g], 
                                                           gb.GRB.LESS_EQUAL,
                                                           generation_capacity[g], name='day-ahead dispatch and reserved capacity upper bound')
    
    # reformulation of chance-constrained DA_dispatch_max_constraint for wind generators (Eq. 42)
    # Genskrives til at tilpasse vores ligning i rapporten
    k = 0
    for g in WIND_GENERATORS:
        DA_dispatch_max_constraint[g] = DA_model.addLConstr(generator_dispatch[g] + generator_reserve_up[g] 
                                                           + norm.ppf(1-epsilon) * wind_availability_standard_deviation[k] * generation_capacity[g], 
                                                           gb.GRB.LESS_EQUAL,
                                                           wind_availability_expected[k] * generation_capacity[g], name='day-ahead dispatch and reserved capacity upper bound')
        k+=1

    #reformulation of robust RT_balance_constriant (Eq. 43)      
    RT_balance_constraint_intersect = DA_model.addLConstr(
            gb.quicksum(generator_adjustment_up_intersect[g] - generator_adjustment_down_intersect[g] for g in GENERATORS),
            gb.GRB.EQUAL,
            0,name='real-time balance equation (intercept of LDR)')

    RT_balance_constraint = []

    for w in range(n_wf):
        RT_balance_constraint.append(
            DA_model.addLConstr(gb.quicksum(generator_adjustment_up[w][g] - generator_adjustment_down[w][g] for g in GENERATORS),
            gb.GRB.EQUAL,
            0,name='real-time balance equation (slope of LDR associated with wind power of WF%d)' % w)
        )
    

    # reformualtion of chance-constrained adjustment_up_min_constraint (Eq. 44a & 46a)
    adjustement_up_min_constraint_1 = {g:DA_model.addQConstr(gb.quicksum(x_1[w][g]**2 for w in range(n_wf)),
                                                       gb.GRB.LESS_EQUAL,
                                                       y[0][g]**2,name='chance-constrained reformualtion of adjustment_up_min_constraint 1/2') for g in GENERATORS} 

    # fortegns fejl?
    adjustement_up_min_constraint_2 = {g:DA_model.addLConstr(y[0][g], 
                                                       gb.GRB.EQUAL,
                                                       generator_adjustment_up_intersect[g]
                                                       + gb.quicksum(wind_availability_expected[w] * generator_adjustment_up[w][g] for w in range(n_wf)),
                                                       name='chance-constrained reformualtion of adjustment_up_min_constraint 2/2') for g in GENERATORS}
    
    adjustement_up_min_constraint = []
    for w in range(n_wf):
        adjustement_up_min_constraint.append({g:DA_model.addLConstr(x_1[w][g], 
                                                       gb.GRB.EQUAL,
                                                       norm.ppf(1-epsilon) * wind_availability_standard_deviation[w] * generator_adjustment_up[w][g],
                                                       name='chance-constrained reformualtion of adjustment_up_min_constraint 2/2') for g in GENERATORS} 
        )
    
    # reformualtion of chance-constrained adjustment_down_min_constraint (Eq. 45a and 47a)
    adjustement_down_min_constraint_1 = {g:DA_model.addQConstr(gb.quicksum(x_2[w][g]**2 for w in range(n_wf)), 
                                                       gb.GRB.LESS_EQUAL,
                                                       y[1][g]**2,name='chance-constrained reformualtion of adjustment_down_min_constraint 1/2') for g in GENERATORS} 

    adjustement_down_min_constraint_2 = {g:DA_model.addLConstr(y[1][g], 
                                                       gb.GRB.EQUAL,
                                                       generator_adjustment_down_intersect[g]
                                                       + gb.quicksum(wind_availability_expected[w] * generator_adjustment_down[w][g] for w in range(n_wf)),
                                                       name='chance-constrained reformualtion of adjustment_down_min_constraint 2/2') for g in GENERATORS} 

    adjustement_down_min_constraint = []
    for w in range(n_wf):
        adjustement_down_min_constraint.append({g:DA_model.addLConstr(x_2[w][g], 
                                                       gb.GRB.EQUAL,
                                                       norm.ppf(1-epsilon) * wind_availability_standard_deviation[w] * generator_adjustment_down[w][g],
                                                       name='chance-constrained reformualtion of adjustment_down_min_constraint 2/2') for g in GENERATORS} 
        )

    # reformualtion of chance-constrained adjustment_up_max_constraint (Eq. 44b and 46b)
    adjustement_up_max_constraint_1 = {g:DA_model.addQConstr(gb.quicksum(x_3[w][g]**2 for w in range(n_wf)), 
                                                       gb.GRB.LESS_EQUAL,
                                                       y[2][g]**2,name='chance-constrained reformualtion of adjustment_up_max_constraint 1/2') for g in GENERATORS} 

    adjustement_up_max_constraint_2 = {g:DA_model.addLConstr(y[2][g], 
                                                       gb.GRB.EQUAL,
                                                       generator_reserve_up[g]-generator_adjustment_up_intersect[g]
                                                       - gb.quicksum(wind_availability_expected[w] * generator_adjustment_up[w][g] for w in range(n_wf)),
                                                       name='chance-constrained reformualtion of adjustment_up_max_constraint 2/2') for g in GENERATORS} 

    adjustement_up_max_constraint = []
    for w in range(n_wf):
        adjustement_up_max_constraint.append({g:DA_model.addLConstr(x_3[w][g], 
                                                       gb.GRB.EQUAL,
                                                       norm.ppf(1-epsilon) * wind_availability_standard_deviation[w] * generator_adjustment_up[w][g],
                                                       name='chance-constrained reformualtion of adjustment_up_max_constraint 2/2') for g in GENERATORS} 
        )

    # reformualtion of chance-constrained adjustment_down_max_constraint (Eq. 45b and 47b)
    adjustement_down_max_constraint_1 = {g:DA_model.addQConstr(gb.quicksum(x_4[w][g]**2 for w in range(n_wf)), 
                                                       gb.GRB.LESS_EQUAL,
                                                       y[3][g]**2,name='chance-constrained reformualtion of adjustment_down_max_constraint 1/2') for g in GENERATORS} 

    adjustement_down_max_constraint_2 = {g:DA_model.addLConstr(y[3][g], 
                                                       gb.GRB.EQUAL,
                                                       generator_reserve_down[g] - generator_adjustment_down_intersect[g]
                                                       - gb.quicksum(wind_availability_expected[w] * generator_adjustment_down[w][g] for w in range(n_wf)),
                                                       name='chance-constrained reformualtion of adjustment_down_max_constraint 2/2') for g in GENERATORS} 

    adjustement_down_max_constraint = []
    for w in range(n_wf):
        adjustement_down_max_constraint.append({g:DA_model.addLConstr(x_4[w][g], 
                                                       gb.GRB.EQUAL,
                                                       norm.ppf(1-epsilon) * wind_availability_standard_deviation[w] * generator_adjustment_down[w][g],
                                                       name='chance-constrained reformualtion of adjustment_down_max_constraint 2/2') for g in GENERATORS} 
        )

    # reformulation of chance-constrained RT_min_production_constraint (Eq. 48a)
    RT_min_production_constraint_1 = {g:DA_model.addQConstr(gb.quicksum(x_5[w][g]**2 for w in range(n_wf)), 
                                                       gb.GRB.LESS_EQUAL,
                                                       y[4][g]**2,name='chance-constrained reformualtion of RT_min_production_constraint 1/2') for g in GENERATORS} 

    RT_min_production_constraint_2 = {g:DA_model.addLConstr(y[4][g], 
                                                       gb.GRB.EQUAL,
                                                       generator_dispatch[g] + generator_adjustment_up_intersect[g] - generator_adjustment_down_intersect[g]
                                                       + gb.quicksum(wind_availability_expected[w] * (generator_adjustment_up[w][g] - generator_adjustment_down[w][g]) for w in range(n_wf)),
                                                        name='chance-constrained reformualtion of RT_min_production_constraint 2/2') for g in GENERATORS} 
    RT_min_production_constraint = []
    for w in range(n_wf):
        RT_min_production_constraint.append({g:DA_model.addLConstr(x_5[w][g], 
                                                       gb.GRB.EQUAL,
                                                       norm.ppf(1-epsilon)*wind_availability_standard_deviation[w]*(generator_adjustment_up[w][g]-generator_adjustment_down[w][g]),
                                                       name='chance-constrained reformualtion of adjustment_up_min_constraint 2/2') for g in GENERATORS}  
        )

    # reformulation of chance-constrained RT_max_production_constraint (Eq. 48b and 49b)
    RT_max_production_constraint_1 = {}
    RT_max_production_constraint_2 = {}
    RT_max_production_constraint_3 = {}
    
    
    for g in CONTROLLABLE_GENERATORS:
        RT_max_production_constraint_1[g] = DA_model.addQConstr(gb.quicksum(x_6[w][g]**2 for w in range(n_wf)), 
                                                           gb.GRB.LESS_EQUAL,
                                                           y[5][g]**2,name='chance-constrained reformualtion of RT_min_production_constraint 1/2') 

        RT_max_production_constraint_2[g] = DA_model.addLConstr(y[5][g], 
                                                           gb.GRB.EQUAL,
                                                           generation_capacity[g]-generator_dispatch[g]
                                                           -generator_adjustment_up_intersect[g]+generator_adjustment_down_intersect[g]
                                                           -gb.quicksum(wind_availability_expected[w] * (generator_adjustment_up[w][g] - generator_adjustment_down[w][g]) for w in range(n_wf)),
                                                           name='chance-constrained reformualtion of RT_min_production_constraint 2/2') 

    RT_max_production_constraint = []
    for w in range(n_wf):
        RT_max_production_constraint.append({g:DA_model.addLConstr(x_6[w][g], 
                                                        gb.GRB.EQUAL,
                                                        norm.ppf(1-epsilon)*wind_availability_standard_deviation[w]*(generator_adjustment_up[w][g]-generator_adjustment_down[w][g]),
                                                        name='chance-constrained reformualtion of adjustment_up_max_constraint 2/2') for g in CONTROLLABLE_GENERATORS}  
        )


    k=0
    for g in WIND_GENERATORS:

        RT_max_production_constraint_1[g] = DA_model.addQConstr(gb.quicksum(x_7[w][g]**2 for w in range(n_wf)), 
                                                    gb.GRB.LESS_EQUAL,
                                                    y[5][g]**2,name='chance-constrained reformualtion of RT_min_production_constraint 1/2') 


        RT_max_production_constraint_2[g] = DA_model.addLConstr(y[5][g], 
                                                    gb.GRB.EQUAL,
                                                    -generator_dispatch[g] - generator_adjustment_up_intersect[g] + generator_adjustment_down_intersect[g]
                                                    -gb.quicksum(wind_availability_expected[w] * (generator_adjustment_up[k][g] - generator_adjustment_down[k][g]) for w in range(n_wf))
                                                    -wind_availability_expected[k] * generation_capacity[g],
                                                    name='chance-constrained reformualtion of RT_min_production_constraint 2/2') 
        for w in range(n_wf):
            if w == k:
                rhs = norm.ppf(1-epsilon)*wind_availability_standard_deviation[w]*(generator_adjustment_up[w][g]
                                                                                -generator_adjustment_down[w][g]
                                                                                -generation_capacity[g])
            else:
                rhs = norm.ppf(1-epsilon)*wind_availability_standard_deviation[w]*(generator_adjustment_up[w][g]
                                                                                -generator_adjustment_down[w][g])
                                                                                                                    
            RT_max_production_constraint_3[g] = DA_model.addLConstr(x_7[w][g], 
                                                    gb.GRB.EQUAL,
                                                        rhs,
                                                        name='chance-constrained reformualtion of RT_min_production_constraint 1/2') 
        k+=1

    # optimize ED problem (primal)
    DA_model.optimize()
    
    optimal_DA_objval = DA_model.ObjVal
    optimal_DA_cost = sum(dispatch_cost[g]*generator_dispatch[g].x for g in GENERATORS)
    optimal_reserve_cost = sum(reserve_cost_up[g]*generator_reserve_up[g].x + reserve_cost_down[g]*generator_reserve_down[g].x for g in GENERATORS)
    optimal_DA_dispatch = {g:generator_dispatch[g].x for g in GENERATORS}
    optimal_reserve_up = {g:generator_reserve_up[g].x for g in GENERATORS}
    optimal_reserve_down = {g:generator_reserve_down[g].x for g in GENERATORS}
    
    return optimal_DA_objval, optimal_DA_cost, optimal_reserve_cost, optimal_DA_dispatch, optimal_reserve_up, optimal_reserve_down


optimal_DA_objval, optimal_DA_cost, optimal_reserve_cost, optimal_DA_dispatch, optimal_reserve_up, optimal_reserve_down = _solve_reserve_dimensioning_model_()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-28
Set parameter TimeLimit to value 500
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: AMD Ryzen 7 7800X3D 8-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 800 rows, 1062 columns and 2982 nonzeros
Model fingerprint: 0x8f58f8c1
Model has 108 quadratic constraints
Coefficient statistics:
  Matrix range     [7e-03, 2e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e-01, 3e+01]
  Bounds range     [3e+01, 6e+02]
  RHS range        [2e+00, 2e+03]
Presolve removed 6 rows and 6 columns
Presolve time: 0.00s
Presolved: 794 rows, 1056 columns, 2964 nonzeros
Presolved model has 108 second-order cone constraints
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 252
 AA' NZ     : 7.425e+03
 Factor NZ  : 1.722e+04 (roughly 1 MB of memory)
 Factor Ops : 4.030e+05 (less